In [111]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
import scipy as scp

In [112]:
def COPT_function_old(Cgens,Ugens,P_round):
    print("Inside COPT_function, Cgens, Ugens, P_round: ", len(Cgens), len(Ugens), P_round)
    # print(Cgens)
    # print(Ugens)
    # assert Ugens has no negative values
    assert np.all(np.array(Ugens) >= 0)
    # Eduardo Jerez, based on Bart Tuinema's Matlab code
    # COPT calculation
    # COPT = COPT_function(Cgens,Ugens,P_round)
    # Cgens = unit capacities
    # Ugens = Generator unavailabilities (FOR's)
    # P_round = capacities are rounded to this value

    Cgens = np.multiply(P_round,np.round(np.divide(Cgens,P_round)))  # round capacities
    P_total = sum(Cgens)  # total capacity
    n_units = len(Cgens)  # number of units

    COPT = np.zeros([np.int_(P_total/P_round), 2])
    COPT[:,0] = range(0,np.int_(P_total),P_round)
    COPT[0,1] = 1
    for i in range(n_units):
        COPT2 = np.multiply(COPT[:,1],Ugens[i])  # COPT when unit i is off
        COPT[:,1] = np.multiply(COPT[:,1],(1-Ugens[i]))  # COPT when unit i is on
        COPT[:,1] = np.add(COPT[:,1], np.hstack([np.zeros([np.int_(Cgens[i]/P_round)]), COPT2[0:-np.int_((Cgens[i]/P_round))]]))    #COPT2 is shifted down by the capacity of the generator

    #COPT = [COPT zeros(size(COPT[0]),1)]    # adding 3rd column
    #COPT(1,3) = 1
    # print("Shape of transposed COPT:", np.transpose(COPT).shape)
    # print("Shape of subtracted array:", np.subtract(1, np.hstack([[0], np.cumsum(COPT[0:-1,1])])).shape)
    
    COPT = np.transpose(np.vstack([np.transpose(COPT), np.subtract(1,np.hstack([[0],np.cumsum(COPT[0:-1,1])]))]))
    # Convert to DataFrame and round probabilities to 4 decimals
    COPT = pd.DataFrame(COPT, columns=['Capacity', 'Probability', 'Cumulative Probability'])
    # COPT['Probability'] = COPT['Probability'].round(4)
    # COPT['Cumulative Probability'] = COPT['Cumulative Probability'].round(4)
    return COPT

In [113]:
def generate_COPT(Cgens, Ugens, P_round=50):
    # Round the capacities to the nearest P_round
    Cgens = np.multiply(P_round, np.round(np.divide(Cgens, P_round)))
    
    total_capacity = sum(Cgens)
    n_steps = int(total_capacity // P_round + 1)
    
    # Initialize probabilities with zeros
    probabilities = np.zeros(n_steps)
    probabilities[0] = 1  # 100% chance of 0 MW out
    
    for i in range(len(Cgens)):
        new_probabilities = probabilities.copy()
        for j in range(n_steps):
            if j - int(Cgens[i]//P_round) >= 0:
                # Probability of this step being out considering the generator being unavailable
                new_probabilities[j] += probabilities[j - int(Cgens[i]//P_round)] * Ugens[i]
            # Probability of this step being out considering the generator being available
            new_probabilities[j] *= (1 - Ugens[i])
        probabilities = new_probabilities
    
    # Create COPT table
    COPT = pd.DataFrame({
        'Capacity': [i*P_round for i in range(n_steps)],
        'Probability': probabilities,
        'Cumulative Probability': 1 - np.cumsum(probabilities) + probabilities
    })
    
    return COPT

In [114]:
# Capacity and Unavailability 
df = pd.read_csv('./data/Cgens_Ugens.csv')
# Capacity of generators
Cgens = df['Cgens'].values.tolist()
# Probability of failure of generators (unavailability)
Ugens = df['Ugens'].values.tolist()
# Wind DF
df = pd.read_csv('./data/load_Pwind.csv')
# Hourly load
NLh_load = df['NLh_load'].values.tolist()
# Power of first offshore wind scenario
P_off_wind1 = df['P_off_wind1'].values.tolist()
# Power of second offshore wind scenario
P_off_wind2 = df['P_off_wind2'].values.tolist()
# Power of third offshore wind scenario
P_off_wind3 = df['P_off_wind3'].values.tolist()
# Round to a multiple of this (50)
P_round = 50

testCgens = [449.0, 600.0, 339.0, 450.0, 454.0, 209.0, 800.0, 120.0, 332.0, 249.0, 800.0, 419.0, 630.0, 780.0, 450.0, 220.0, 520.0, 600.0, 403.0, 335.0, 600.0, 640.0, 265.0, 638.0, 800.0, 224.0, 130.0, 645.0, 800.0, 415.0, 335.0, 361.0, 780.0, 520.0, 430.0, 650.0, 400.0, 695.0, 400.0, 600.0, 335.0, 602.0, 332.0, 101.0, 459.0, 350.0, 115.0, 121.0, 335.0, 350.0, 599.0, 253.0, 231.0, 335.0, 500.0, 240.0, 249.0, 25.0, 75.0, 125.0, 175.0, 225.0, 275.0, 325.0, 375.0, 425.0, 475.0, 525.0, 575.0, 625.0, 675.0, 725.0, 775.0, 825.0, 875.0, 925.0, 975.0]
testUgens = [0.0614, 0.0899, 0.0874, 0.07, 0.07, 0.0899, 0.07, 0.0874, 0.0899, 0.0874, 0.07, 0.0696, 0.0899, 0.0874, 0.07, 0.0874, 0.0899, 0.07, 0.0899, 0.0696, 0.07, 0.08, 0.0874, 0.08, 0.07, 0.0899, 0.07, 0.0899, 0.07, 0.0696, 0.0696, 0.0899, 0.0874, 0.0899, 0.07, 0.07, 0.0874, 0.0899, 0.0874, 0.07, 0.0696, 0.0899, 0.0899, 0.0899, 0.0899, 0.0696, 0.08, 0.0632, 0.0696, 0.07, 0.0874, 0.0899, 0.0874, 0.0696, 0.089, 0.0874, 0.0874, 0.8823059360730594, 0.923972602739726, 0.9360730593607306, 0.943607305936073, 0.9468036529680366, 0.9526255707762558, 0.9642694063926941, 0.9665525114155251, 0.9687214611872146, 0.9691780821917808, 0.9722602739726027, 0.9713470319634703, 0.9684931506849315, 0.9728310502283105, 0.9705479452054795, 0.970662100456621, 0.969634703196347, 0.9707762557077626, 0.959931506849315, 0.8194063926940639]
Cgens = testCgens
Ugens = testUgens
df_COPT_A = generate_COPT(Cgens, Ugens, 50)


In [115]:
df_COPT_B = COPT_function_old(Cgens[:57], Ugens[:57], 50)
# Plot Probabilty of A and B vs Capacity
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_COPT_A['Capacity'], y=df_COPT_A['Probability'], name='A'))
fig.add_trace(go.Scatter(x=df_COPT_B['Capacity'], y=df_COPT_B['Probability'], name='B'))
fig.update_layout(title='Probability vs Capacity', xaxis_title='Capacity (MW)', yaxis_title='Probability')
# Size 800x800
fig.update_layout(width=800,height=800)
fig.show()


Inside COPT_function, Cgens, Ugens, P_round:  57 57 50
